In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 786.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.

In [2]:
from fastkaggle import *
from fastai.vision.all import *
from fasterai.sparse.all import *
from torchvision.models import vgg16_bn

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


time: 4.08 s (started: 2023-05-28 06:28:03 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 11.7 s (started: 2023-05-28 06:28:07 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ), 
                                   bs=32//2)

time: 9.07 s (started: 2023-05-28 06:28:19 +00:00)


In [5]:
def run_once(func):
    """
    A decorator that runs a function only once and returns the result for all subsequent calls.
    """
    memo = {}
    def wrapper(*args, **kwargs):
        if func not in memo:
            memo[func] = func(*args, **kwargs)
        return memo[func]
    return wrapper


time: 687 µs (started: 2023-05-28 06:28:28 +00:00)


In [6]:
@run_once
def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size 
    print('Model size: {} Byte'.format(model_size))


time: 562 µs (started: 2023-05-28 06:28:29 +00:00)


In [7]:
class MemoryCallback(Callback):
    def before_batch(self):
        get_model_file_size(self.learn.model)

time: 499 µs (started: 2023-05-28 06:28:29 +00:00)


In [8]:
sp_cb=[
      GradientAccumulation(32), 
      SparsifyCallback(sparsity=25, 
                         granularity='filter', 
                         context='local', 
                         criteria=large_final, 
                         schedule=one_cycle
                         ),
      MemoryCallback(),
]

time: 722 µs (started: 2023-05-28 06:28:29 +00:00)


In [9]:
model = vgg16_bn(num_classes=10)

time: 2.13 s (started: 2023-05-28 06:28:29 +00:00)


In [10]:
learn = Learner(dls, 
                model, 
                metrics=[accuracy,
                         error_rate,
                        ]).to_fp16()
learn.unfreeze()
learn.recorder.train_metrics = True

time: 7.2 ms (started: 2023-05-28 06:28:31 +00:00)


In [11]:
learn.fit_one_cycle(1, 3e-5, cbs=sp_cb)


Pruning of filter until a sparsity of [25]%
Saving Weights at epoch 0


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.150732,0.204167,0.795833,2.046952,0.284135,0.715865,01:24


Model size: 537239848 Byte
Sparsity at the end of epoch 0: [25.0]%
Final Sparsity: [25.0]%
Sparsity in Conv2d 2: 25.00%
Sparsity in Conv2d 5: 25.00%
Sparsity in Conv2d 9: 25.00%
Sparsity in Conv2d 12: 25.00%
Sparsity in Conv2d 16: 25.00%
Sparsity in Conv2d 19: 25.00%
Sparsity in Conv2d 22: 25.00%
Sparsity in Conv2d 26: 25.00%
Sparsity in Conv2d 29: 25.00%
Sparsity in Conv2d 32: 25.00%
Sparsity in Conv2d 36: 25.00%
Sparsity in Conv2d 39: 25.00%
Sparsity in Conv2d 42: 25.00%
time: 1min 24s (started: 2023-05-28 06:28:32 +00:00)
